In [1]:
from huggingface_hub import login
login()


In [2]:
# Colab Notebook for Fine-tuning LLMs on Different Use Cases

## Install Required Packages
!pip install transformers datasets accelerate peft bitsandbytes trl einops

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from trl import SFTTrainer
import torch
import os

# Set PyTorch CUDA Memory Allocation Config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Function to load model and tokenizer
def load_model(model_name, use_4bit=True):
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_4bit=use_4bit, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Ensure tokenizer has a pad token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

# Select model
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
model, tokenizer = load_model(MODEL_NAME, use_4bit=False)  # Disable 4-bit quantization

# Load dataset
def load_finetune_data(task_type, sample_size=500):  # Reduce dataset size
    if task_type == "classification":
        dataset = load_dataset("ag_news")
    elif task_type == "chat":
        dataset = load_dataset("tatsu-lab/alpaca")
    elif task_type == "multi_dataset":
        dataset1 = load_dataset("ag_news")
        dataset2 = load_dataset("tatsu-lab/alpaca")
        dataset = dataset1["train"].train_test_split(test_size=0.1)
        dataset = dataset["train"].add_item(dataset2["train"])
    else:
        raise ValueError("Invalid task type")

    # Reduce dataset size for training
    dataset["train"] = dataset["train"].select(range(min(sample_size, len(dataset["train"]))))
    if "test" in dataset:
        dataset["test"] = dataset["test"].select(range(min(sample_size // 10, len(dataset["test"]))))

    return dataset

# Select dataset
task_type = "chat"  # Change this to "classification" or "multi_dataset"
dataset = load_finetune_data(task_type, sample_size=500)

# LORA Configuration
peft_config = LoraConfig(
    r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Reduce batch size to prevent OOM
    per_device_eval_batch_size=1,
    num_train_epochs=1,  # Reduce epochs for quicker testing
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    bf16=True,  # Enable mixed precision training to save memory
    gradient_accumulation_steps=4  # Simulate larger batch size
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"] if "test" in dataset else None,
    args=training_args,
    tokenizer=tokenizer,
)

trainer.train()

# Save Model
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.686300
20,1.290900
30,1.075100
40,1.014300
50,1.067900
60,0.976300
70,1.032100
80,0.994300
90,0.938700
100,1.008900


('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/tokenizer.model',
 './finetuned_model/added_tokens.json',
 './finetuned_model/tokenizer.json')